In [1]:
    import tensorflow as tf
    # include path
    print(tf.sysconfig.get_include())
    # library path 
    print(tf.sysconfig.get_lib())

/home/az/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/az/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


/home/az/anaconda3/lib/python3.6/site-packages/tensorflow/include
/home/az/anaconda3/lib/python3.6/site-packages/tensorflow


In [2]:
import tensorflow as tf
CUDA_VISIBLE_DEVICES=0
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))
#python3 -c 'import tensorflow as tf; print(tf.test.is_gpu_available())'


b'Hello, TensorFlow!'


In [3]:
print(tf.test.is_gpu_available())
print(tf.test.gpu_device_name())

True
/device:GPU:0


In [4]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8203134522506823368, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 155582464
 locality {
   bus_id: 1
 }
 incarnation: 17780709368865929169
 physical_device_desc: "device: 0, name: GeForce RTX 2060 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [5]:
tf.test.is_built_with_cuda()

True

In [6]:
import argparse
import math
from datetime import datetime
#import h5pyprovider
import numpy as np
import tensorflow as tf
import socket
import importlib
import matplotlib.pylab as plt
import os
import pickle
# os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES']="1"

import sys
BASE_DIR = os.path.abspath('')
print(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(BASE_DIR) # model
sys.path.append(os.path.join(BASE_DIR, 'tf_utils'))
import provider
import tf_util

/home/az/Documents/D-FCN


In [7]:
# Argparses
EPOCH_CNT = 0

BATCH_SIZE = 6
NUM_POINT = 2048
MAX_EPOCH = 200
BASE_LEARNING_RATE = .01
# GPU_INDEX = 7
GPU_INDEX = 1
MOMENTUM = 0.9
OPTIMIZER = 'adam'
DECAY_STEP = 20000
DECAY_RATE = 0.5

In [8]:
MODEL = importlib.import_module('DFCN_pointnet2_group2') # import network module
MODEL_FILE = os.path.join(BASE_DIR, 'DFCN_pointnet2_group2'+'.py')
LOG_DIR = 'log_wen_v16_sample8192_group2_lw14_F1_noheight'
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
os.system('cp models/DFCN_pointnet2.py %s' % (LOG_DIR)) # bkp of model def
os.system('cp tf_utils/DFCN_util_xy2.py %s' % (LOG_DIR)) # bkp of model def
# os.system('cp train_pointsift_lx_npsplit_V16.ipynb %s' % (LOG_DIR)) # bkp of train procedure
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')
# LOG_FOUT.write(str(FLAGS)+'\n')

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

HOSTNAME = socket.gethostname()

NUM_CLASSES = 9

BASE_DIR /home/az/Documents/D-FCN/tf_utils


In [9]:
def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)

def get_learning_rate(batch):
    learning_rate = tf.train.exponential_decay(
                        BASE_LEARNING_RATE,  # Base learning rate.
                        batch * BATCH_SIZE,  # Current index into the dataset.
                        DECAY_STEP,          # Decay step.
                        DECAY_RATE,          # Decay rate.
                        staircase=True)
    learning_rate = tf.maximum(learning_rate, 0.00001) # CLIP THE LEARNING RATE!!
    return learning_rate        

def get_bn_decay(batch):
    bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*BATCH_SIZE,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
    bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
    return bn_decay

def Acc_from_confusions(confusions):
    
    TP = np.diagonal(confusions, axis1=-2, axis2=-1)
    TP_plus_FN = np.sum(confusions, axis=-1)
    TP_plus_FP = np.sum(confusions, axis=-2)
    
    mAcc = np.sum(TP)/np.sum(confusions)
    
    precision = TP / (TP_plus_FP + 1e-6)
    recall = TP / (TP_plus_FN+ 1e-6)
    fscore = 2*(precision * recall)/(precision + recall + 1e-6)
    
    ave_F1 = np.mean(fscore)
    
    s = 'Overall accuracy：{:5.2f}  Average F1 score：{:5.2f} \n'.format(100 * mAcc, 100 * ave_F1)
    s += log_acc(precision)
    s += log_acc(recall)
    s += log_acc(fscore)
    
    log_string(s)
    
    return mAcc, ave_F1
    

def log_acc(acc_list):
    s = ""
    for acc in acc_list:
        s += '{:5.2f} '.format(100 * acc)
    s += '\n'
    return s

def pc_normalize(pc):
    l = pc.shape[0]
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc
    
def drawPlot(x,y,name):
    plt.rcParams['savefig.dpi'] = 300 
    plt.plot(np.arange(0,len(x)),x,'k-',alpha=1,label='Train max: '+str(round(max(x),3))+', min: '+str(round(min(x),3)))
    plt.plot(np.arange(0,len(y)),y,'r-',alpha=1,label='Test max: '+str(round(max(y),3))+', min: '+str(round(min(y),3)))
    plt.legend()
    plt.xlabel('epoch',fontsize=9)
    plt.ylabel(name+' value',fontsize=9)
    plt.savefig(LOG_DIR+"/"+name+".png",bbox_inches='tight')
    plt.show()
    
def drawF1Plot(x,name):
    plt.rcParams['savefig.dpi'] = 300 
    plt.plot(np.arange(0,len(x)),x,'k-',alpha=1,label='Train max: '+str(round(max(x),3))+', min: '+str(round(min(x),3)))
    plt.legend()
    plt.xlabel('epoch',fontsize=9)
    plt.ylabel(name+' value',fontsize=9)
    plt.savefig(LOG_DIR+"/"+name+".png",bbox_inches='tight')
    plt.show()
    
    
def placeholder_inputs(batch_size, num_point):
    pointclouds_pl = tf.placeholder(tf.float32, shape=(None, None, 3))
    labels_pl = tf.placeholder(tf.int32, shape=(None, None))
    smpws_pl = tf.placeholder(tf.float32, shape=(None, None))
    return pointclouds_pl, labels_pl, smpws_pl

def pc_normalize_min_max(data):
    mindata = np.min(data[:,:3], axis=0)
    maxdata = np.max(data[:,:3], axis=0)
    return 2*(data[:,:3] - mindata)/(maxdata - mindata)

def pc_normalize_min(data):
    mindata = np.min(data[:,:3], axis=0)
    # MS: Function now returns mindata here
    return (data[:,:3] - mindata), mindata


def get_batch(dataset, index, npoints = NUM_POINT):
  
    if(dataset =='train'):
        cub_l = 30000000.0
        cub_w = 30000000.0
        cub_h = 100000000.0
        point_set =  trainSet[:,:3] - np.min(trainSet[:,:3], axis=0)
        semantic_seg = trainSet[:,4].astype(np.int32)
        coordmax = np.max(point_set,axis=0)
        coordmin = np.min(point_set,axis=0)
        smpmin = np.maximum(coordmax-[cub_l,cub_w,cub_h], coordmin)
        smpmin[2] = coordmin[2]
        smpsz = np.minimum(coordmax-smpmin,[cub_l,cub_w,cub_h])
        smpsz[2] = coordmax[2]-coordmin[2]
        isvalid = False
        for i in range(10):
            curcenter = point_set[np.random.choice(len(semantic_seg),1)[0],:]
            curmin = curcenter-[cub_l/2,cub_w/2,cub_h/2]
            curmax = curcenter+[cub_l/2,cub_w/2,cub_h/2]
            curmin[2] = coordmin[2]
            curmax[2] = coordmax[2]
            curchoice = np.sum((point_set>=(curmin-0.0))*(point_set<(curmax+0.0)),axis=1)==3
            cur_point_set = point_set[curchoice,:]
            cur_semantic_seg = semantic_seg[curchoice]
            cur_feat_set = trainFeats[curchoice,:]
    #         if len(cur_semantic_seg)<npoints:
            if len(cur_semantic_seg)==0:
                continue
            mask = np.sum((cur_point_set>=(curmin-0.0))*(cur_point_set<(curmax+0.0)),axis=1)==3
            vidx = np.ceil((cur_point_set[mask,:2]-curmin[:2])/(curmax[:2]-curmin[:2])*[31.0,31.0])
            vidx = np.unique(vidx[:,0]*31.0+vidx[:,1])
            isvalid = np.sum(cur_semantic_seg>-1)/1.0/len(cur_semantic_seg)>=0.7 and len(vidx)/31.0/31.0>=0.3
#             print('isvalid', isvalid,len(vidx)/31.0/31.0,np.sum(cur_semantic_seg>-1),len(cur_semantic_seg))
            if isvalid:
                break
        choice = np.random.choice(len(cur_semantic_seg), npoints, replace=True)
        point_set = cur_point_set[choice,:]
        feature_set = cur_feat_set[choice,:]
        semantic_seg = cur_semantic_seg[choice]
        mask = mask[choice]
        sample_weight = labelweights[semantic_seg]
        sample_weight *= mask
        return point_set, semantic_seg, sample_weight,feature_set
    
    if(dataset =='test'):
        ##original 30/30/100
        #100 - 1000 = 500
        #300000
        #1000000
        cub_l = 30000000.0
        cub_w = 30000000.0
        cub_h = 100000000.0
        #all rows, 0&1&2 columns = xyz 
        point_set =  testSet[:,:3] - np.min(testSet[:,:3], axis=0)#minimum in column?/ min of x,y,z
        #5th column = classification, used for batch label for corretness
        semantic_seg = testSet[:,4].astype(np.int32)
        coordmax = np.max(point_set,axis=0)
        coordmin = np.min(point_set,axis=0)
        smpmin = np.maximum(coordmax-[cub_l,cub_w,cub_h], coordmin) #cub covers much more pts in our dataset
        smpmin[2] = coordmin[2]#y?
        smpsz = np.minimum(coordmax-smpmin,[cub_l,cub_w,cub_h])
        smpsz[2] = coordmax[2]-coordmin[2]
        isvalid = False
        for i in range(10):
            curcenter = point_set[np.random.choice(len(semantic_seg),1)[0],:]
            curmin = curcenter-[cub_l/2,cub_w/2,cub_h/2]
            curmax = curcenter+[cub_l/2,cub_w/2,cub_h/2] 
            curmin[2] = coordmin[2]
            curmax[2] = coordmax[2]
            curchoice = np.sum((point_set>=(curmin-0.0))*(point_set<(curmax+0.0)),axis=1)==3
            cur_point_set = point_set[curchoice,:]#!
            cur_semantic_seg = semantic_seg[curchoice]#
            cur_feat_set = testFeats[curchoice,:]
    #         if len(cur_semantic_seg)<npoints:
            if len(cur_semantic_seg)==0:
                continue
            mask = np.sum((cur_point_set>=(curmin-0.0))*(cur_point_set<(curmax+0.0)),axis=1)==3
            vidx = np.ceil((cur_point_set[mask,:2]-curmin[:2])/(curmax[:2]-curmin[:2])*[31.0,31.0])
            vidx = np.unique(vidx[:,0]*31.0+vidx[:,1])
            isvalid = np.sum(cur_semantic_seg>-1)/1.0/len(cur_semantic_seg)>=0.7 and len(vidx)/31.0/31.0>=0.3
#             print('isvalid', isvalid,len(vidx)/31.0/31.0,np.sum(cur_semantic_seg>-1),len(cur_semantic_seg))
            if isvalid:
                break
            
        choice = np.random.choice(len(cur_semantic_seg), npoints, replace=True) #npoints= 2048 = repeat
        point_set = cur_point_set[choice,:]#! repeat #choose x,y,z, n rows 
        feature_set = cur_feat_set[choice,:]
        semantic_seg = cur_semantic_seg[choice]
        mask = mask[choice]
        sample_weight = labelweights[semantic_seg]
        sample_weight *= mask
        return point_sets, semantic_segs, sample_weights,feature_set
    
def get_batch_wdp(dataset, batch_idx):
    bsize = BATCH_SIZE
    batch_data = np.zeros((bsize, NUM_POINT, 3))
    ps_xyz_min = np.zeros((bsize, 1, 3))
    ps_out = np.zeros((bsize, NUM_POINT, 3))
    
    batch_feats = np.zeros((bsize, NUM_POINT, 1))
    batch_label = np.zeros((bsize, NUM_POINT), dtype=np.int32)
    batch_smpw = np.zeros((bsize, NUM_POINT), dtype=np.float32)
    for i in range(bsize):
        ps,seg,smpw,feat = get_batch('train',index=0)
        #ps_out[i,...] = ps
        ps,ps_min = pc_normalize_min(ps)
        batch_data[i,...] = ps
        batch_label[i,:] = seg
        batch_smpw[i,:] = smpw
        batch_feats[i,:] = feat

        dropout_ratio = np.random.random()*0.875 # 0-0.875
        drop_idx = np.where(np.random.random((ps.shape[0]))<=dropout_ratio)[0]
        batch_data[i,drop_idx,:] = batch_data[i,0,:]
        batch_label[i,drop_idx] = batch_label[i,0]
        batch_smpw[i,drop_idx] *= 0
        #ps_xyz_min[i,...] = ps_min
        
    return batch_data, batch_label, batch_smpw, batch_feats

def get_batch_wdp_test(dataset, batch_idx):
    bsize = BATCH_SIZE
    batch_data = np.zeros((bsize, NUM_POINT, 3))
    ps_xyz_min = np.zeros((bsize, 1, 3))
    ps_out = np.zeros((bsize, NUM_POINT, 3))

    batch_feats = np.zeros((bsize, NUM_POINT, 1))
    batch_label = np.zeros((bsize, NUM_POINT), dtype=np.int32)
    batch_smpw = np.zeros((bsize, NUM_POINT), dtype=np.float32)
    for i in range(bsize):#6
        #point_set, semantic_seg, sample_weight,feature_set,cur_semantic_seg#cur_semantic_seg
        ps,seg,smpw,feat,cur_semantic_seg = get_batch('test',index=0)#
        # ps_out collects the pre-normalized batch_data
        ps_out[i,...] = ps
        # ps_min is collected not from pc_normalize_min
        ps,ps_min = pc_normalize_min(ps)
        batch_data[i,...] = ps
        batch_label[i,:] = seg
        batch_smpw[i,:] = smpw
        batch_feats[i,:] = feat

        dropout_ratio = np.random.random()*0.875 # 0-0.875
        drop_idx = np.where(np.random.random((ps.shape[0]))<=dropout_ratio)[0]
        batch_data[i,drop_idx,:] = batch_data[i,0,:]
        batch_label[i,drop_idx] = batch_label[i,0]
        batch_smpw[i,drop_idx] *= 0
    	# ps_xyz_min collects the xyz minimums vector for each batch
        ps_xyz_min[i,...] = ps_min
    # ps_xyz_min and px_out are now returned.
    #cur_semantic_seg
    return batch_data, batch_label, batch_smpw, batch_feats, ps_xyz_min, ps_out,cur_semantic_seg

In [10]:

# train_f = open('Data/train_merge_min_norm_fea.pickle', 'rb')
# train_xyz, train_label, train_feats = pickle.load(train_f, encoding='bytes')
# train_xyz, train_label, train_feats = pickle.load(train_f)
# train_f.close()

testSet = np.loadtxt('Data/03labeled_seg.pts',skiprows=1)
label_w = testSet[:,4].astype('uint8')
if (testSet[:,3].max() != 0):
    testSet[:,3] = testSet[:,3]/testSet[:,3].max() #height above ground
if (testSet[:,5].max() != 0):
    testSet[:,5] = testSet[:,5]/testSet[:,5].max() #reflectance
    
testFeats = testSet[:,5:6] #only use reflectance

NUM_CLASSES = 9
label_values = range(NUM_CLASSES)

#trainSet = np.loadtxt('Data/train_bk.pts',skiprows=1)
trainSet = np.loadtxt('Data/03labeled_seg.pts',skiprows=1)

label_w = trainSet[:,4].astype('uint8')
trainSet[:,3] = trainSet[:,3]/trainSet[:,3].max() #height above ground
trainSet[:,5] = trainSet[:,5]/trainSet[:,5].max() #reflectance

# trainFeats = trainSet[:,[3,5]] #use reflectance and height above ground
trainFeats = trainSet[:,5:6] #only use reflectance

labelweights = np.zeros(9)
tmp,_ = np.histogram(label_w,range(10))
labelweights = tmp
labelweights = labelweights.astype(np.float32)
labelweights = labelweights/np.sum(labelweights)
labelweights = 1/np.log(1.4+labelweights)
print(labelweights)

labelweights_t = np.ones(9)
print(labelweights_t)

[1.1561419 2.9720137 2.9720137 2.8228025 2.9720137 2.9720137 2.9720137
 2.9720137 2.9720137]
[1. 1. 1. 1. 1. 1. 1. 1. 1.]


/home/az/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


In [11]:
def train_one_epoch(sess, ops, train_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    
    log_string('----')
    
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    
    
   # Shuffle train samples
    train_idxs = np.arange(0, len(train_xyz))
    np.random.shuffle(train_idxs)
    num_batches = len(train_xyz)//BATCH_SIZE
    
    for batch_idx in range(num_batches):
        
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE
        
        batch_data, batch_label, batch_smpw, batch_feats = get_batch_wdp('train', batch_idx)
        
        if batch_idx % (num_batches/2) == 0:
            print('Current batch/total batch num: %d/%d'%(batch_idx,num_batches))
        
        aug_data = provider.rotate_point_cloud_z(batch_data)
        
        feed_dict = {ops['pointclouds_pl']: aug_data,
                     ops['feature_pl']: batch_feats,
                     ops['labels_pl']: batch_label,
                     ops['smpws_pl']: batch_smpw,
                     ops['is_training_pl']: is_training,}
        summary, step, _, loss_val, pred_val, lr_val = sess.run([ops['merged'], ops['step'], ops['train_op'], 
                                                                 ops['loss'], ops['pred'], ops['learnrate']],
                                         feed_dict=feed_dict)
        #summary, step, _, loss_val, pred_val, lr_val = sess.run([ops['merged'], ops['step'], ops['train_op'], 
        #                                                         ops['loss'], ops['pred'], ops['learnrate']],
                                         #                        feed_dict=feed_dict)
        train_writer.add_summary(summary, step)
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == batch_label)
        total_correct += correct
        total_seen += (BATCH_SIZE*NUM_POINT)
        loss_sum += loss_val
        
    log_string('learn rate: %f' % (lr_val))
    log_string('mean loss: %f' % (loss_sum / float(num_batches)))
    log_string('accuracy: %f' % (total_correct / float(total_seen)))
    
    mloss = loss_sum / float(num_batches)
    macc = total_correct / float(total_seen)
    return mloss, macc

def eval_one_epoch_whole_scene(sess, ops, test_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
    
    log_string('----')
    
    test_idxs = np.arange(0, len(test_xyz))
    # Shuffle train samples
    np.random.shuffle(test_idxs)
    num_batches = len(test_xyz)//BATCH_SIZE
    
    #TEST_BATCH_SIZE = 1
    #num_batches = len(test_xyz)
    
    Confs = []
    
    #
    #Confs_normalized = []
    #pred_list = []
    
    is_continue_batch = False
    
    for batch_idx in range(num_batches):
        
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE
        #cur_semantic_seg
        #batch_data, batch_label, batch_smpw, batch_feats, ps_xyz_min, ps_out,cur_semantic_seg = get_batch_wdp_test('test', batch_idx)
        batch_data, batch_label, batch_smpw, batch_feats = get_batch_wdp_test('test', batch_idx)
        if batch_idx % (num_batches/2) == 0:
            print('Current batch/total batch num: %d/%d'%(batch_idx,num_batches))

            
        aug_data, rotation_matrix = provider.rotate_point_cloud_z(batch_data)
        
        feed_dict = {ops['pointclouds_pl']: aug_data,
                     ops['feature_pl']: batch_feats,
                     ops['labels_pl']: batch_label,
                     ops['smpws_pl']: batch_smpw,
                     ops['is_training_pl']: is_training}
        print("feed_dict: ",feed_dict)

        summary, step, _, loss_val, pred_val, lr_val = sess.run([ops['merged'], ops['step'], ops['train_op'], ops['loss'], ops['pred'], ops['learnrate']],
                                         feed_dict=feed_dict)
        
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == batch_label)
        #pred_list.append([ps_out, pred_val])
        
        total_correct += correct
        total_seen += (BATCH_SIZE*NUM_POINT)
        loss_sum += loss_val
        
       
        #NUM_POINT_fact = batch_data.shape[1]
        NUM_POINT_fact = (BATCH_SIZE*NUM_POINT)
        for i in range(BATCH_SIZE):
            for j in range(NUM_POINT):
                l = batch_label[i, j]
                total_seen_class[l] += 1
                total_correct_class[l] += (pred_val[i, j] == l)
                
        from sklearn.metrics import confusion_matrix
        Confs += [confusion_matrix(batch_label.flatten(), pred_val.flatten(), label_values)]
        #add normalize = true
        #x/total true viechles
        Confs_normalized += [confusion_matrix(batch_label.flatten(), pred_val.flatten(), label_values, normalize='true')]
    
    #end of for loop
    C = np.sum(np.stack(Confs), axis=0).astype(np.float32)
    #C_normalized = np.sum(np.stack(Confs_normalized), axis=0).astype(np.float32)
    oa, avgF1 = Acc_from_confusions(C)
   
    log_string('learn rate: %f' % (lr_val))
    log_string('eval mean loss: %f' % (loss_sum / float(num_batches)))
    log_string('eval accuracy: %f'% (total_correct / float(total_seen)))
    log_string('eval avg class acc: %f' % (np.mean(np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float))))
    
    mloss = loss_sum / float(num_batches)
    macc = total_correct / float(total_seen)
    return oa, avgF1

In [12]:
pointclouds_pl, labels_pl, smpws_pl = placeholder_inputs(BATCH_SIZE, NUM_POINT)
feature_pl = tf.placeholder(tf.float32, shape=(None, None, 1))
is_training_pl = tf.placeholder(tf.bool, shape=())

# Note the global_step=batch parameter to minimize. 
# That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
batch = tf.Variable(0)
bn_decay = get_bn_decay(batch)
tf.summary.scalar('bn_decay', bn_decay)

# Get model and loss 
pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl, NUM_CLASSES, bn_decay=bn_decay, feature=feature_pl)
loss = MODEL.get_loss(pred, labels_pl, smpws_pl)

tf.summary.scalar('loss', loss)

correct = tf.equal(tf.argmax(pred, 2), tf.to_int64(labels_pl))
accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(BATCH_SIZE*NUM_POINT)
tf.summary.scalar('accuracy', accuracy)

# Get training operator
learning_rate = get_learning_rate(batch)
tf.summary.scalar('learning_rate', learning_rate)
if OPTIMIZER == 'momentum':
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=MOMENTUM)
elif OPTIMIZER == 'adam':
    optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss, global_step=batch)

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

In [13]:
## Create a session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = True
sess = tf.Session(config=config)
print(tf.test.is_gpu_available())

True


In [14]:
## Add summary writers
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'),
                          sess.graph)
test_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'))
print(tf.test.is_gpu_available())

True


In [15]:
## Init variables
init = tf.global_variables_initializer()
sess.run(init, {is_training_pl:True})
print(tf.test.is_gpu_available())

True


In [16]:
##
ops = {'pointclouds_pl': pointclouds_pl,
       'labels_pl': labels_pl,
       'feature_pl': feature_pl,
       'smpws_pl': smpws_pl,
       'is_training_pl': is_training_pl,
       'pred': pred,
       'loss': loss,
       'train_op': train_op,
       'merged': merged,
       'step': batch,
       'learnrate': learning_rate}
print(tf.test.is_gpu_available())

True


In [17]:
##
train_acc_list=[]
test_acc_list=[]
train_loss_list=[]
test_F1_list=[]
print(tf.test.is_gpu_available())

True


In [13]:
# Create a session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = True
sess = tf.Session(config=config)

# Add summary writers
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'),
                          sess.graph)
test_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'))

# Init variables
init = tf.global_variables_initializer()
sess.run(init, {is_training_pl:True})

ops = {'pointclouds_pl': pointclouds_pl,
       'labels_pl': labels_pl,
       'feature_pl': feature_pl,
       'smpws_pl': smpws_pl,
       'is_training_pl': is_training_pl,
       'pred': pred,
       'loss': loss,
       'train_op': train_op,
       'merged': merged,
       'step': batch,
       'learnrate': learning_rate}

train_acc_list=[]
test_acc_list=[]
train_loss_list=[]
test_F1_list=[]

In [14]:
train_xyz,train_label,train_weights,train_feats = [],[],[],[]
num_train_samples = 1000
for i in range(num_train_samples):
    train_point_set, train_semantic_seg, train_sample_weight, train_feature_set = get_batch('train',0,
                                                                                            npoints=NUM_POINT)
    train_xyz.append(train_point_set)
    train_label.append(train_semantic_seg)
    train_weights.append(train_sample_weight)
    train_feats.append(train_feature_set)

In [15]:
best_F1 = -1

for epoch in range(MAX_EPOCH):
    log_string('**** EPOCH %03d ****' % (epoch))
    sys.stdout.flush()

    train_loss, train_acc = train_one_epoch(sess, ops, train_writer)
    test_acc, test_F1 = eval_one_epoch_whole_scene(sess, ops, test_writer)

    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)

    test_F1_list.append(test_F1)

    
    drawF1Plot(test_F1_list, "F1_score")
    
    drawPlot(train_acc_list,test_acc_list,"Accuracy")

    # Save the variables to disk.
    
    if test_F1 > best_F1:
        best_F1 = test_F1
        save_path = saver.save(sess, os.path.join(LOG_DIR, "best_model_epoch_%03d.ckpt"%(epoch)))
        log_string("Model saved in file: %s" % save_path)
                
    if epoch % 10 == 0:
        save_path = saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"))
        log_string("Model saved in file: %s" % save_path)

**** EPOCH 000 ****
----
Current batch/total batch num: 0/166


ValueError: setting an array element with a sequence.

In [13]:
eval_one_epoch_whole_scene(sess, ops, test_writer)

----
Overall accuracy：82.04  Average F1 score：67.59 
75.72 85.10 89.90 92.91 76.83 95.44 60.07 34.02 75.29 
66.00 74.98 92.73 55.80 13.04 91.75 60.12 52.91 84.54 
70.53 79.72 91.29 69.72 22.30 93.56 60.10 41.41 79.65 

learn rate: 0.000020
eval mean loss: 0.635362
eval accuracy: 0.820402
eval avg class acc: 0.657640


(0.8204022, 0.67585766)

In [12]:
ops

{'pointclouds_pl': <tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>,
 'labels_pl': <tf.Tensor 'Placeholder_1:0' shape=(?, ?) dtype=int32>,
 'feature_pl': <tf.Tensor 'Placeholder_3:0' shape=(?, ?, 1) dtype=float32>,
 'smpws_pl': <tf.Tensor 'Placeholder_2:0' shape=(?, ?) dtype=float32>,
 'is_training_pl': <tf.Tensor 'Placeholder_4:0' shape=() dtype=bool>,
 'pred': <tf.Tensor 'fc2/BiasAdd:0' shape=(?, ?, 9) dtype=float32>,
 'loss': <tf.Tensor 'sparse_softmax_cross_entropy_loss/value:0' shape=() dtype=float32>,
 'train_op': <tf.Operation 'Adam' type=AssignAdd>,
 'merged': <tf.Tensor 'Merge/MergeSummary:0' shape=() dtype=string>,
 'step': <tf.Variable 'Variable:0' shape=() dtype=int32_ref>,
 'learnrate': <tf.Tensor 'Maximum:0' shape=() dtype=float32>}

In [18]:
from keras import backend as K
K.clear_session()
save_path = saver.save(sess, os.path.join(LOG_DIR, "best_model_cleared.ckpt"))

Using TensorFlow backend.
